# 3c Create Routed Paths TomTom

The goal of the following notebook is to compute the Routed Paths from a Mobility Demand using TomTom as the routing algorithm.
___
We translate the $N$ trips in the Mobility Demand $D$ into $N$ paths obtaining a multiset $\overline{D}$ of routed paths within the urban environment. 
Each path $P_v(e_o,e_d, R) {=} (e_o, \dots, e_d)$ of a vehicle $v$
is a sequence of edges on the road network connecting $e_o$ and $e_d$, obtained by some routing algorithm $R$. 
When a vehicle's path is generated by a routing algorithm $R$, we say that the vehicle is $R$-routed.
In $\overline{D} = {\{P_1, \dots, P_N\}}$, the routed paths are generated independently by (different) routing algorithms.
___
## TomTom routing API

For obtaining a **key** you need to register at https://developer.tomtom.com/user/register.

In the Free plan, you have **2500 free requests daily**. Once registered, on the web profile there is a counter of the daily requests made (the "free non-tile requests" in the "Activity" section).

See https://developer.tomtom.com/routing-api/documentation/routing/common-routing-parameters#section-1-common-routing-parameters for the routing parameters that affect the routing algorithm.

The **most important parameter** is `routeType`, which specifies the type of optimization used when calculating routes (can be `fastest`, `shortest`, `short`, `eco`, `thrilling`). Default is `fastest`.
- `fastest`: Route calculation is optimized by travel time, while keeping the routes sensible. For example, the calculation may avoid shortcuts along inconvenient side roads or long detours that only save very little time.
- `shortest`: Route calculation is optimized by travel distance, while keeping the routes sensible. For example, straight routes are preferred over those incurring turns.
- `short`: Route calculation is optimized such that a good compromise between small travel time and short travel distance is achieved.
- `eco`: Route calculation is optimized such that a good compromise between small travel time and low fuel or energy consumption is achieved.
- `thrilling`: Route calculation is optimized such that routes include interesting or challenging roads and use as few motorways as possible.

In our analysis we use `short`.

Other parameters are, for example: 
- `travelMode` (`truck`, `taxi`, `bus`, `pedestrian`, ...). Default is `car`.
- `vehicleMaxSpeed`
___

In [1]:
import warnings
warnings.filterwarnings("ignore")

from utils import *
import json
import sumolib

#### Load road network

In [2]:
road_network_path = "../sumo_simulation_data/milan_road_network.net.xml"
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

#### TomTom routing API

In [3]:
tomtom_api_key = "<INSERT-YOUR-KEY-HERE>"

In [4]:
# dict mobility demand path

dict_mobility_demand_path = "../data/dict_mobility_demand.json"

In [5]:
with open(dict_mobility_demand_path) as json_file:
    dict_mobility_demand = json.load(json_file)

In [6]:
vehicle_list_all = [v_id for v_id in dict_mobility_demand.keys() if "vehicle" in v_id]

In [7]:
id_from = 0
id_to = len(vehicle_list_all)

vehicle_list = vehicle_list_all[id_from:id_to]

### Get the routing suggestions from TomTom

In [8]:
dict_results_tomtom = create_routed_demand_tomtom(dict_mobility_demand, vehicle_list, road_network, tomtom_api_key)

### Visualize some suggestions

In [9]:
# the identifier of the vehicle's route we want to visualize

v_id = "vehicle_0"

points_traj = dict_results_tomtom[v_id]['points']

m = folium.Map(location=[points_traj[0][1], points_traj[0][0]], tiles='cartodbpositron', zoom_start=13)

# TomTom Suggestion
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in points_traj], weigth=3, color="blue").add_to(m)
m

### Save the TomTom results into a dictionary

you can merge later several TomTom results with the notebook 3d


In [11]:
# create the folder (if doesn't exist)
if not os.path.exists("../data/dicts_suggestion/"):
    os.makedirs("../data/dicts_suggestion/")

In [10]:
a_file = open("../data/dicts_suggestion/dict_results_tomtom_["+str(id_from)+"_"+str(id_to)+"].json", "w")
json.dump(dict_results_tomtom, a_file)
a_file.close()